## Increasing Computational Efficacy by Smarter Utilization of Memory Associated With Kernel Matrices
### Questions
- [ ] How do we update the cholesky factorization without computing an entirely new decomposition?

### Issues
- [ ] The final subroutine of computing covariances of fantasized gradients against other fantasized gradients needs to be checked. I believe the hessian computation has a sign error somewhere.
- Subroutine 7 has an issue with it's indices. I need to refactor the logic to support comparing the covariances of the current fantasized gradient sample against all previous fantasized gradient samples.

In [1]:
using DataFrames
using LinearAlgebra
using Plots

In [91]:
include("../rollout.jl")
include("../testfns.jl")
include("../testfns.jl")

TestGramacyLee (generic function with 1 method)

In [611]:
include("./CovarianceMatrix.jl")

get_KXX (generic function with 1 method)

In [612]:
m, h, d = 1, 1, 2
psm = PreallocatedSymmetricMatrix{Float64}(m, h, d)
X = rand(d, m+h+1)

2×3 Matrix{Float64}:
 0.701269  0.503061   0.63559
 0.461188  0.0407565  0.690117

In [613]:
# Construct covariance measure
θ = [1.]
ψ = kernel_matern52(θ);
σn2 = 1e-6

1.0e-6

## Building the Matrix via Subroutines

In [614]:
# Subroutine Order: 1, 2->3->4->5->8->6->7
Knews = []
# Subroutine number 1: Knowns against knowns
Kupdate = eval_KXX(ψ, X[:, 1:m]; σn2=σn2)
fKupdate = cholesky(Kupdate).U'
update_knowns!(psm, Kupdate)
cholesky_update_knowns!(psm, fKupdate)

for fndx in m+1:m+h+1
    # Subroutine number 2: Fantasized against knowns
    Kupdate = eval_KXY(ψ, X[:, fndx:fndx], X[:, 1:m])
    update_fantasized_vs_knowns!(psm, Kupdate, fndx)
    cholesky_update_fantasized_vs_knowns!(psm, fndx)
       
    # Subroutine number 3: Fantasized against fantasized
    Kupdate = eval_KXX(ψ, X[:, m+1:fndx]; σn2=σn2)
    update_fantasized_vs_fantasized!(psm, Kupdate, fndx)
    cholesky_update_fantasized_vs_fantasized!(psm, fndx)
    
    # 4 + 5
    # Kupdate = eval_∇KxX(ψ, X[:, fndx], X[:, 1:fndx])
    # update_gradfantasized_vs_fantasized_and_known!(psm, Kupdate, fndx-m) # (fndx - m)th grad
    
    # Subroutine number 4: Fantasized gradients against knowns
    Kupdate = eval_∇KxX(ψ, X[:, fndx], X[:, 1:m])
    update_gradfantasized_vs_known!(psm, Kupdate, fndx-m) # (fndx - m)th grad
    cholesky_update_gradfantasized_vs_known!(psm, fndx-m)
    
    # Subroutine number 5: Fantasized gradients against fantasized gradients
    Kupdate = eval_∇KxX(ψ, X[:, fndx], X[:, m+1:fndx])
    update_gradfantasized_vs_fantasized!(psm, Kupdate, fndx-m)
    # cholesky_update_gradfantasized_vs_fantasized!(psm, fndx-m)
    
    # Subroutine number 8:
    if fndx-m > 1
        Kupdate = -eval_∇KxX(ψ, X[:, fndx], X[:, m+1:fndx-1])
        klength = size(Kupdate, 1) * size(Kupdate,2)
        Kupdate = reshape(Kupdate, klength)
        update_fantasized_vs_allprev_gradfantasized!(psm, Kupdate, fndx-m)
    end
    println("Gradient Index: $(fndx-m)")
    # cholesky_update_fantasized_vs_allprev_gradfantasized!(psm, fndx-m)
    cholesky_update_gradfantasized_vs_fantasized!(psm, fndx-m)
    
    # # Subroutine number 6: Fantasized gradients against self fantasized gradients
    Kupdate = -eval_Hk(ψ, zeros(d))
    update_gradfantasized_vs_self!(psm, Kupdate, fndx-m)
    
    # Subroutine number 7: Fantasized gradients against other fantasized gradients
    Knews = []
    grad_ndx = fndx - m
    
    # Skip first gradient observation since previous fantasized values don't exist yet
    for j in 1:grad_ndx-1 # fndx-m computes the kth gradient ndx starting from 1
        Kn = -eval_Hk(ψ, X[:, fndx] - X[:, fndx - j])
        push!(Knews, Kn)
    end
    
    # Reverse here since we go from current gradient observation to the first
    # To adjust this, fndx - j should just go from the beginning to fndx
    Knews = reverse(Knews)
    if fndx-m > 1
        Kupdate = Knews[1]
        for i = 2:length(Knews)
            Kupdate = hcat(Kupdate, Knews[i])
        end
        
        update_gradfantasized_vs_gradfantasized!(psm, Kupdate, fndx-m)
    end
    cholesky_update_gradfantasized_vs_gradfantasized!(psm, fndx-m)
end

Gradient Index: 1
G = (2, 2), L31 = (2, 1), L32 = (2, 1)
Gradient Index: 2
G = (4, 4), L31 = (4, 1), L32 = (4, 2)


$$
L_{32} = (E - L_{31}L_{21}^T)L_{22}^{-T}
$$

In [617]:
psm.fK[]

7×7 LowerTriangular{Float64, Matrix{Float64}}:
  1.0          ⋅          ⋅           ⋅          ⋅         ⋅         ⋅ 
  0.848642    0.528968    ⋅           ⋅          ⋅         ⋅         ⋅ 
  0.955271   -0.152142   0.253595     ⋅          ⋅         ⋅         ⋅ 
  0.238275   -0.382272  -0.635758    1.02935     ⋅         ⋅         ⋅ 
  0.505419   -0.810862   0.0161544  -0.408147   0.766078   ⋅         ⋅ 
  0.0984935  -0.393479  -0.607082    0.336828  -0.501893  0.876494   ⋅ 
 -0.343305   -0.602933   0.931478    0.272575  -0.101441  0.250238  0.412822

In [618]:
cholesky(psm.K[]).L

7×7 LowerTriangular{Float64, Matrix{Float64}}:
  1.0          ⋅          ⋅           ⋅          ⋅         ⋅         ⋅ 
  0.848642    0.528968    ⋅           ⋅          ⋅         ⋅         ⋅ 
  0.955271   -0.152142   0.253595     ⋅          ⋅         ⋅         ⋅ 
  0.238275   -0.382272  -0.635758    1.02935     ⋅         ⋅         ⋅ 
  0.505419   -0.810862   0.0161544  -0.408147   0.766078   ⋅         ⋅ 
  0.0984935  -0.393479  -0.607082    0.336828  -0.501893  0.876494   ⋅ 
 -0.343305   -0.602933   0.931478    0.272575  -0.101441  0.250238  0.412822

In [619]:
psm.fK[] ≈ cholesky(psm.K[]).L

true

# A Toy Example Problem

In [ ]:
testfn = TestAckley(1)
tplot(testfn)

In [ ]:
# Checking my logic for a single update to a cholesky factorization
Xcur = rand(2, 2)
Xupdate = hcat(Xcur, rand(2, 1))

Kcur = eval_KXX(ψ, Xcur)
Lcur = cholesky(Kcur).U'
Kupdate = eval_KXX(ψ, Xupdate)
Lupdate = cholesky(Kupdate).U'

l31 = Kupdate[3,1] / Lcur[1,1]
l32 = (Kupdate[3, 2] - l31*Lcur[2,1]) / Lcur[2,2]
l33 = sqrt(Kupdate[3,3] - l31^2 - l32^2)

Lother = [Lcur zeros(2, 1);
          l31 l32 l33]

@assert Lother ≈ Lupdate

In [626]:
@allocated Z = rand(1000, 1000)

8000048

In [627]:
sizeof(Z)

8000000

In [624]:
@allocated V = @view Z[10:end, 10:end]

160